In [1]:
from parlai.core.agents import create_agent
from parlai.agents.emely.emely import EmelyAgent
from parlai.core.opt import Opt
from pathlib import Path
import torch
from parlai.utils.io import PathManager
from time import time

model_path = Path.cwd() / 'data/models/blender/blender_90M/'
assert model_path.is_dir()

opt_path = model_path / 'model.opt'
opt = Opt.load(opt_path)

# Change opts
opt['skip_generation'] = False
opt['init_model'] = (model_path / 'model').as_posix()
opt['no_cuda'] = True  # Cloud run doesn't offer gpu support

# Inference options
opt['inference'] = 'greedy' # 'beam'
opt['beam_size'] = 1

emely_agent = EmelyAgent(opt)

with PathManager.open("../../saved_models/emely_scripted_test.pt", "rb") as f:
    scripted_module = torch.jit.load(f)

nruns = 100
text = "Hi Emely, how are you?\nI'm good thanks! What do you do for work?\nI write code and I drink coffe."

16:52:09 | loading dictionary from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model.dict
16:52:10 | num words = 54944
16:52:10 | DEPRECATED: XLM should only be used for backwards compatibility, as it involves a less-stable layernorm operation.
16:52:11 | Total parameters: 87,508,992 (87,508,992 trainable)
16:52:11 | Loading existing model params from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model


/home/ckjellson/code/emely-models/ParlAI/parlai/agents/emely/emely.py:29: UserWarning: Tried to add persona to agent history but EmelyAgent.persona is None
  warnings.warn('Tried to add persona to agent history but EmelyAgent.persona is None')


In [2]:
# Test emely agent
t1 = time()
for i in range(nruns):
    reply = emely_agent.observe_and_act(text)
testtime = time() - t1
print(testtime)
print(reply)

45.27503943443298
that ' s cool ! i ' ve never tried coffee . i ' d love to try it though .


In [2]:
# Test scripted emely agent (if the above cell has been run, restart kernel before running this cell)
t1 = time()
for i in range(nruns):
    reply = scripted_module(text)
testtime = time() - t1
print(testtime)
print(reply)

/home/ckjellson/anaconda3/envs/emelymodels/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return forward_call(*input, **kwargs)


44.12752318382263
that ' s cool ! i ' ve never tried coffee . i ' d love to try it though .
